
<p><img align="left" src="https://www.cqf.com/themes/custom/creode/logo.svg" style="vertical-align: top; padding-top: 23px;" width="10%"/>
<img align="right" src="https://upload.wikimedia.org/wikipedia/commons/c/c3/Python-logo-notext.svg" style="vertical-align: middle;" width="12%"/>
<font color="#306998"><h1><center>Python Labs</center></h1></font></p>
<p></p><h1><center>Value At Risk</center></h1>
<center><b>Kannan Singaravelu</b></center>
<center>kannan.singaravelu@fitchlearning.com</center>



<h2 id="Value-at-Risk">Value-at-Risk<a class="anchor-link" href="#Value-at-Risk">¶</a></h2><p>Value at Risk - <strong>VaR</strong> - is one of the most important metrics that is used to measures the risk associated with a financial position or a portfolio of financial instruments. VaR can be defined as <strong><code>the maximum loss with a confidence level over a predetermined period.</code></strong> Let's say that the 1-day 95% VaR of our portfolio is $\$100$. This means that 95% of the time, it is expected that - under normal market conditions - we will not lose more than $100 by holding our portfolio over one day.</p>
<p>Three approaches that are commonly used in the industry are</p>
<ul>
<li><strong>Parametric</strong></li>
<li><strong>Historical Simulation</strong> </li>
<li><strong>Monte Carlo Simulation</strong></li>
</ul>



<h3 id="Import-Libraries">Import Libraries<a class="anchor-link" href="#Import-Libraries">¶</a></h3>


In [ ]:

import pandas as pd
import numpy as np
from numpy.linalg import multi_dot

from scipy.stats import norm
from tabulate import tabulate

import matplotlib.pyplot as plt
from helper import plot_var 




<h3 id="Retrieve-Data">Retrieve Data<a class="anchor-link" href="#Retrieve-Data">¶</a></h3><p>We will use the FAANG stocks as before to build for calculation of VaR</p>


In [ ]:

# Load locally stored data
df = pd.read_csv('data/faang_stocks.csv', parse_dates=True, index_col=0)['2013':]

# Check first 5 values 
df.head()




<h3 id="Calculate-Returns">Calculate Returns<a class="anchor-link" href="#Calculate-Returns">¶</a></h3>


In [ ]:

# Calculate daily returns
returns = df.pct_change().dropna()



In [ ]:

# Visualize AMZN daily returns
plt.plot(returns['AMZN'], color='orange')
plt.axhline(y=0.10, ls='dotted', color='black')
plt.axhline(y=-0.10, ls='dotted', color='black')
plt.title('AMZN Daily Returns')
plt.grid(True)




<h3 id="Parametric-VaR">Parametric VaR<a class="anchor-link" href="#Parametric-VaR">¶</a></h3><p>The Variance-covariance is a parametric method which assumes (almost always) that the returns are normally distributed. In this method, we first calculate the mean and standard deviation of the returns to derive the risk metric. Based on the assumption of normality, we can generalise,</p>
$$ VaR = position * (\mu_{period} - z * \sigma_p) $$<table>
<thead><tr>
<th style="text-align:left">Confidence Level</th>
<th style="text-align:left">Value At Risk     </th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left"><code>90%</code></td>
<td style="text-align:left">$\mu$ - $1.29$ * $\sigma$ </td>
</tr>
<tr>
<td style="text-align:left"><code>95%</code></td>
<td style="text-align:left">$\mu$ - $1.64$ * $\sigma$             </td>
</tr>
<tr>
<td style="text-align:left"><code>99%</code></td>
<td style="text-align:left">$\mu$ - $2.33$ * $\sigma$             </td>
</tr>
</tbody>
</table>


In [ ]:

# Calculate mean and standard deviation 
mean = np.mean(returns['AMZN'])
stdev = np.std(returns['AMZN'])

# Calculate VaR at difference confidence level
VaR_90 = norm.ppf(1-0.90,mean,stdev)
VaR_95 = norm.ppf(1-0.95,mean,stdev) #norm.ppf(0.05)
VaR_99 = norm.ppf(1-0.99,mean,stdev)



In [ ]:

# Ouput results in tabular format
table = [['90%', VaR_90],['95%', VaR_95],['99%', VaR_99] ]
header = ['Confidence Level', 'Value At Risk']
print(tabulate(table,headers=header))




<p>Lets now define a VaR function so that we can use it calculate it for individual stocks</p>


In [ ]:

# VaR function
def VaR(symbol, cl=0.95):
    mean = np.mean(returns[symbol])
    stdev = np.std(returns[symbol])
    
    return np.around(100*norm.ppf(1-cl,mean,stdev),4)



In [ ]:

# VaR for stocks
print('VaR for FAANG Stocks')
print('---'*11)
[print(f'VaR at 95% CL for {stock:4} : {VaR(stock)}%') for stock in df.columns][0]



In [ ]:

# Visulalize VaR at 95% confidence level
plot_var()




<p>Now, let's assume that we have 1,000 shares of AMZN's stock on April 29, 2020. What is the maximum loss next day with a confidence level of 99%?</p>


In [ ]:

num_of_shares = 1000
price = df['AMZN'].iloc[-1]
position = num_of_shares * price 

amzn_var = position * VaR_99

print(f'Amazon Holding Value: {position}')
print(f'Amazon VaR at 99% confidence level is: {amzn_var}')




<p>VaR can also be calculated using the above formula at 99% confidence level.</p>


In [ ]:

# VaR calculation by appling direct formulae
position * (mean + norm.ppf(1-0.99) * stdev)         # mean-2.33*stdev




<h3 id="Scaling-VaR">Scaling VaR<a class="anchor-link" href="#Scaling-VaR">¶</a></h3><p>Now, let's calculate VaR over a 5-day period. To scale it, multiply by square root of time.</p>
$$ VaR = position * (\mu_{period} - z * \sigma_p) * \sqrt{T}$$


In [ ]:

forecast_days = 5
f_VaR_90 = VaR_90*np.sqrt(forecast_days)
f_VaR_95 = VaR_95*np.sqrt(forecast_days)
f_VaR_99 = VaR_99*np.sqrt(forecast_days)



In [ ]:

ftable = [['90%', f_VaR_90],['95%', f_VaR_95],['99%', f_VaR_99] ]
fheader = ['Confidence Level', '5-Day Forecast Value At Risk']
print(tabulate(ftable,headers=fheader))




<p>Let's now calculate AMZN VaR over a 5-day period with a confidence level of 99%</p>


In [ ]:

amzn_var_5days = position * f_VaR_99

print(f'Amazon Holding Value: {position}')
print(f'Amazon VaR at 99% confidence level is: {amzn_var_5days}')



In [ ]:

# Scaled VaR over different time horizon
plt.figure(figsize=(8,6))
plt.plot(range(100),[-100*VaR_95*np.sqrt(x) for x in range(100)])
plt.xlabel('Horizon')
plt.ylabel('Var 95 (%)')
plt.title('VaR_95 Scaled by Time');




<h3 id="Historical-VaR">Historical VaR<a class="anchor-link" href="#Historical-VaR">¶</a></h3><p>Asset returns do not necessarily follow a normal distribution. An alternative is to use sorted returns to evaluate a VaR. This method uses historical data where returns are sorted in ascending order to calculate maximum possible loss for a given confidence level.</p>


In [ ]:

# Use quantile function for Historical VaR
hVaR_90 = returns['AMZN'].quantile(0.10)
hVaR_95 = returns['AMZN'].quantile(0.05)
hVaR_99 = returns['AMZN'].quantile(0.01)



In [ ]:

htable = [['90%', hVaR_90],['95%', hVaR_95],['99%', hVaR_99]]
print(tabulate(htable,headers=header))




<h3 id="Expected-Short-Fall">Expected Short Fall<a class="anchor-link" href="#Expected-Short-Fall">¶</a></h3><p>VaR is a reasonable measure of risk if assumption of normality holds. Else, we might underestimate the risk if we observe a fat tail or overestimate the risk if tail is thinner. Expected shortfall or Conditional Value at Risk - <strong>CVaR</strong> - is an estimate of expected shortfall sustained in the worst 1 - x% of scenarios. It is defined as the average loss based on the returns that are lower than the VaR threshold. Assume that we have <code>n</code> return observations, then the expected shortfall is</p>
$$ CVaR = \frac 1 n * \sum_{i=1}^{n} R_i[R&lt;hVaR_{cl}]$$<p>where, $R$ is returns, $hVaR$ is historical VaR and $cl$ is the confidence level.</p>


In [ ]:

# Calculate CVar
CVaR_90 = returns['AMZN'][returns['AMZN']<=hVaR_90].mean()
CVaR_95 = returns['AMZN'][returns['AMZN']<=hVaR_95].mean()
CVaR_99 = returns['AMZN'][returns['AMZN']<=hVaR_99].mean()



In [ ]:

ctable = [['90%', CVaR_90],['95%', CVaR_95],['99%', CVaR_99] ]
cheader = ['Confidence Level', 'Conditional Value At Risk']
print(tabulate(ctable,headers=cheader))




<h3 id="MonteCarlo-VaR">MonteCarlo VaR<a class="anchor-link" href="#MonteCarlo-VaR">¶</a></h3><p>The Monte Carlo simulation approach has a number of similarities to historical simulation. It allows us to use actual historical distributions rather than having to assume normal returns. As returns are assumed to follow a normal distribution, we could generate <em><code>n</code></em> simulated returns with the same mean and standard deviation (derived from the daily returns) and then sorted in ascending order to calculate maximum possible loss for a given confidence level.</p>


In [ ]:

# Set seed for reproducibility
np.random.seed(12345)

# Number of simulations
n_sims = 5000

# Simulate returns and sort
sim_returns = np.random.normal(mean, stdev, n_sims)

# Use percentile function for MCVaR
MCVaR_90 = np.percentile(sim_returns,10)
MCVaR_95 = np.percentile(sim_returns, 5)
MCVaR_99 = np.percentile(sim_returns,1)



In [ ]:

mctable = [['90%', MCVaR_90],['95%', MCVaR_95],['99%', MCVaR_99]]
print(tabulate(mctable,headers=header))




<h2 id="Portfolio-VaR">Portfolio VaR<a class="anchor-link" href="#Portfolio-VaR">¶</a></h2>



<h3 id="Assign-Weights">Assign Weights<a class="anchor-link" href="#Assign-Weights">¶</a></h3>


In [ ]:

# Assign random weights
# np.random.seed(40)
# wts = np.random.random((5,1))
wts = np.array([0.2,0.2,0.2,0.2,0.2])[:,np.newaxis]
wts




<h3 id="Portfolio-return">Portfolio return<a class="anchor-link" href="#Portfolio-return">¶</a></h3>


In [ ]:

# Stock returns
returns[:5]



In [ ]:

port_ret = np.dot(returns,wts)
port_ret.flatten()



In [ ]:

port_mean = port_ret.mean()
port_mean




<h3 id="Portfolio-Volatility">Portfolio Volatility<a class="anchor-link" href="#Portfolio-Volatility">¶</a></h3>


In [ ]:

# Covariance matrix
returns.cov()



In [ ]:

# Portfolio volatility
port_stdev = np.sqrt(multi_dot([wts.T, returns.cov(), wts]))
port_stdev.flatten()[0]




<h3 id="Portfolio-Position">Portfolio Position<a class="anchor-link" href="#Portfolio-Position">¶</a></h3>


In [ ]:

# Portfolio Position
port_pos = (df.iloc[-1] * num_of_shares).sum()
port_pos



In [ ]:

# Calculate Portfolio VaR at difference confidence level
pVaR_90 = norm.ppf(1-0.90,port_mean,port_stdev).flatten()[0]
pVaR_95 = norm.ppf(1-0.95,port_mean,port_stdev).flatten()[0]
pVaR_99 = norm.ppf(1-0.99,port_mean,port_stdev).flatten()[0]



In [ ]:

pVaR_95




<h3 id="Portfolio-VaR">Portfolio VaR<a class="anchor-link" href="#Portfolio-VaR">¶</a></h3>


In [ ]:

# Ouput results in tabular format
ptable = [['90%', pVaR_90],['95%', pVaR_95],['99%', pVaR_99]]
header = ['Confidence Level', 'Value At Risk']
print(tabulate(ptable,headers=header))




<p>Let's now compare the portfolio VaR numbers with that of the individual stocks</p>


In [ ]:

# Iterate over symbols
for stock in df.columns:
    pos = df[stock].iloc[-1] * num_of_shares
    pvar = pos * VaR(stock)
    
    print(f'{stock} Holding Value: {pos:0.4}') 
    print(f'{stock} VaR at 95% confidence level: {pvar:0.4}')
    print()

print(f'Portfolio Holding Value: {port_pos:0.4}')
print(f'Portoflio VaR at 95% confidence level is: {port_pos * pVaR_95:0.4}')




<p>The VaR for the current portfolio of $\$ 4.6$ million is \$109,356, which is much lesser than the individual VaR numbers. This signifies the effect of diversification by selecting different stocks.</p>



<h1 id="References">References<a class="anchor-link" href="#References">¶</a></h1><ul>
<li><p>Numpy documentation <a href="https://docs.scipy.org/doc/numpy/">https://docs.scipy.org/doc/numpy/</a></p>
</li>
<li><p>Scipy documentation <a href="https://docs.scipy.org/doc/scipy/reference/">https://docs.scipy.org/doc/scipy/reference/</a></p>
</li>
</ul>
